<h2>Deep Convolutional Generative Adversarial Nets (DCGAN) - generate MNIST image samples</h2>

<p>Original DCGAN paper: <a href='https://arxiv.org/pdf/1511.06434.pdf'>https://arxiv.org/pdf/1511.06434.pdf</a></p>

<p>The model structure is borrowed from and modified based on Arthur Juliani's implementation: <a href='https://medium.com/@awjuliani/generative-adversarial-networks-explained-with-a-classic-spongebob-squarepants-episode-54deab2fce39#.4kvc6juwc'>https://medium.com/@awjuliani/generative-adversarial-networks-explained-with-a-classic-spongebob-squarepants-episode-54deab2fce39#.4kvc6juwc</a></p>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.examples.tutorials.mnist import input_data
import os
import scipy.misc
import time
from images2gif import writeGif
from PIL import Image

## Network Config

In [2]:
tf.reset_default_graph()
np.random.seed(1) 
tf.set_random_seed(1)

# config
pre_train_epochs = 1 # num of epochs to pre-train discriminator
training_epochs = 4001 # num of epochs to train discriminator and generator jointly
batch_size = 40 # batch size for training the model, must be larger than batch_size_sample

# for the generated images
batch_size_sample = 36 # number of images to be generated for viewing the training progress
# the generated images will be concatenated, the product of the following must equal to batch_size_sample
sample_num_rows = 6 # how many rows to have
sample_num_columns = 6 # how many columns to have

leakiness = 0.2 # degree of leakiness used in leaky ReLU
alpha = 0.0002 # base learning rate
beta1 = 0.5 # the fraction factor used in the first momentum term from Adam optimizer
k = 1 # this is number of times to update discriminator for every time generator is updated in each epoch
logs_path = "/tmp/dcgan_mnist" # directory to save the training log to
train_sample_directory = './dcgan_mnist/train_sample/' # directory to save the generated images during training
model_directory = './dcgan_mnist/models' # directory to save trained model
sample_directory = './dcgan_mnist/generated_sample/' # directory to save the generated images

## Load MNIST data

In [ ]:
# download and read the MNIST data
# Each value represent a pixel intensity ranges from 0 - 1 
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

## Declare helpful functions

In [3]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def leaky_relu(x, leak=leakiness, name="leaky_relu"):
    with tf.variable_scope(name):
        return tf.maximum(leak*x, x)

# get a batch of real images, change size from 28*28 to 32*32, return the processed batch
def get_x(batch_size):
    x,_ = mnist.train.next_batch(batch_size) # values range from [0, 1]
    x = (np.reshape(x,[batch_size,28,28,1]) - 0.5) * 2.0 # change value range to [-1, 1] for better leaky relu activation

    # Add padding to the images so they are 32*32
    # (no change in batch_size, add image width by 2 on the left and 2 on the right, add image hight by 2 on the top
    # and 2 on the bottom, no change in image channel)
    # padding values are -1
    x = np.lib.pad(x, ((0,0),(2,2),(2,2),(0,0)),'constant', constant_values=(-1, -1)) 

    return x
        
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow

# wrapper function
def save_images(images, size, image_path):
    images = inverse_transform(images) # rescale image pixel values from [-1, 1] to [0, 1]
    concat_img = concat(images, size) # concatenate individual images to a grid of images
    return scipy.misc.imsave(image_path, concat_img)

# change values from [-1, 1] to [0, 1]
def inverse_transform(images):
    return (images+1.)/2.

# concatenate individual images to a grid of images
def concat(images, size):
    # get height and width for a single image generated, e.g. 32 * 32
    height, width = images.shape[1], images.shape[2] 
    
    # placeholder for a concatenated img, which have 
    # img_height = image height * num images vertically, e.g. 32 * 5
    # img_width = image width * num images horizontally, e.g. 32 * 5
    img = np.zeros((height * size[0], width * size[1])) 

    # loop through each image
    for index, image in enumerate(images):
        j = index / size[0] # image row index
        i = index % size[1] # image column index
        img[j*height:j*height+height, i*width:i*width+width] = image

    return img

### Save orginal images

In [ ]:
if not os.path.exists(train_sample_directory):
    os.makedirs(train_sample_directory)

batch = mnist.train.next_batch(batch_size_sample)[0] # get a sample batch of real images
batch = np.reshape(batch,[batch_size_sample,28,28]) # reshape images
save_images(images=batch, size=[sample_num_rows,sample_num_columns], image_path=train_sample_directory+'/fig_original.png')

## Define the Generative Adversarial Network

#### Generator network

In [4]:
# The generator network takes a noise vector z and return a 32 * 32 image. 
# We use tensorflow slim to perform fractionally-strided convolution, batch normalization and ReLU activation
# for each layer except the last layer
def generator(z):
    
    # Turn z into the first tensor + batch norm + relu
    # Creates a fully connected weight matrix, which is multiplied by the mini-batch of z vectors to 
    # produce a hidden layer with 4*4*64 hidden nodes
    zP = slim.fully_connected(inputs=z,num_outputs=4*4*64,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_init',weights_initializer=initializer)
    # Transform the flat 4*4*64 layer into a tensor, whose kernel size is 4*4 and 64 kernels in total
    # -1 stands for the mini-batch size that's to be computed
    zCon = tf.reshape(zP,[-1,4,4,64])
    
    # Perform fractionally-strided convolution/deconvolution + batch norm + relu
    # num_outputs: number of output filters/activation maps
    # kernel_size: [kernel_height, kernel_width]
    # stride: [stride_height, stride_width], in this case it is generating four pixels out of every pixel, this gives fractionally-strided convolution
    # output size batch_size*8*8*32
    gen1 = slim.convolution2d_transpose(\
        inputs=zCon,num_outputs=32,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    
    # output size batch_size*16*16*16
    gen2 = slim.convolution2d_transpose(\
        inputs=gen1,num_outputs=16,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    
    # output size batch_size*32*32*1
    g_out = slim.convolution2d_transpose(\
        inputs=gen2,num_outputs=1,kernel_size=[5,5],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_output', weights_initializer=initializer)
    
    return g_out

### Discriminator network

In [5]:
# The discriminator network takes a 32*32 image and return a probability of whether it is real or generated
# We use tensorflow slim to perform standard convolution, batch normalization and leaky ReLU activation
# for each layer
def discriminator(image, reuse=False):
    # Creates 16 4*4 filters to convolve on the mini-batch of 32*32*1 images, also perform batch norm + leaky ReLU activation
    # Note that no pooling is performed
    # stride here calculates one pixel out of every 2*2 pixels, this gives strided convolution that will shrink the image as a substitute for pooling
    # Set reuse=True allows discriminator to evaluate both real samples and generated samples 
    # Outputs size batch_size*16*16*16 
    dis1 = slim.convolution2d(inputs=image,num_outputs=16,kernel_size=[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=leaky_relu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    
    # outputs size batch_size*8*8*32 
    dis2 = slim.convolution2d(inputs=dis1,num_outputs=32,kernel_size=[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=leaky_relu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    
    # outputs size batch_size*4*4*64 
    dis3 = slim.convolution2d(inputs=dis2,num_outputs=64,kernel_size=[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=leaky_relu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    
    # flatten the tensor to [batch_size, 4*4*64]
    dis_flat = slim.flatten(dis3)
    
    # create a fully connect layer with dis_flat and just one node in the output layer
    # note there's no batch normalization at this layer
    # outputs size batch_size*1 
    d_out = slim.fully_connected(inputs=dis_flat,num_outputs=1,\
        activation_fn=tf.nn.sigmoid, reuse=reuse, scope='d_output', weights_initializer=initializer)
    
    return d_out

## Connecting two networks together

In [ ]:
z_size = 100 # size of initial noise vector that will be used for generator

# initialize all parameters of the networks
# weights were initialized from a zero-centered Normal distribution with standard deviation 0.02
# tf.truncated_normal returns random values from a normal distribution and made sure no value exceeds 2 std
initializer = tf.truncated_normal_initializer(stddev=0.02)

# placeholders for inputs into the generator and discriminator, respectively.
z_vector = tf.placeholder(shape=[batch_size,z_size],dtype=tf.float32, name='z_vectors') 
x_vector = tf.placeholder(shape=[batch_size,32,32,1],dtype=tf.float32, name='real_images') 

# ---- Pre-training ----
# the discriminator should output probability=1 for all the training images
train_labels=tf.constant(1.0,shape=(batch_size,1), name='pre_train_labels')
# feed images to the discriminator and return the predicted probability
d_pre_output = discriminator(x_vector)
summary_pre_d_x_hist = tf.histogram_summary("pre_train_d_prob_x", d_pre_output)

d_pre_loss=tf.reduce_mean(tf.square(d_pre_output-train_labels))
summary_pre_d_loss = tf.scalar_summary("pre_train_d_loss", d_pre_loss)
# ---- end of Pre-training ----


# ---- DCGAN ----
g_output = generator(z_vector) # generated mini-batch of images from noisy z vectors
d_output_x = discriminator(x_vector,reuse=True) # probabilities for real images
d_output_x = tf.maximum(tf.minimum(d_output_x, 0.99), 0.01) # avoid inf and -inf
summary_d_x_hist = tf.histogram_summary("d_prob_x", d_output_x)

d_output_z = discriminator(g_output,reuse=True) # probabilities for generated images
d_output_z = tf.maximum(tf.minimum(d_output_z, 0.99), 0.01) # avoid inf and -inf
summary_d_z_hist = tf.histogram_summary("d_prob_z", d_output_z)

d_loss = -tf.reduce_mean(tf.log(d_output_x) + tf.log(1-d_output_z)) # loss for discriminator
summary_d_loss = tf.scalar_summary("d_loss", d_loss)

g_loss = -tf.reduce_mean(tf.log(d_output_z)) # loss for generator
summary_g_loss = tf.scalar_summary("g_loss", g_loss)

# the following parameter indices may change if the network structure changes
para_d = tf.trainable_variables()[:7] # parameters for discriminator
para_g = tf.trainable_variables()[7:] # parameters for generator

# only update parameters in discriminator during pre-training
pre_optimizer = tf.train.AdamOptimizer(learning_rate=alpha,beta1=beta1).minimize(d_pre_loss,var_list=para_d)
# only update the weights for the discriminator network
optimizer_op_d = tf.train.AdamOptimizer(learning_rate=alpha,beta1=beta1).minimize(d_loss,var_list=para_d)
# only update the weights for the generator network
optimizer_op_g = tf.train.AdamOptimizer(learning_rate=alpha,beta1=beta1).minimize(g_loss,var_list=para_g)

## Train the DCGAN model

In [ ]:
# create a log folder and save the graph structure, do this before training
g_writer = tf.train.SummaryWriter(logs_path + '/generator', graph=tf.get_default_graph())
d_writer = tf.train.SummaryWriter(logs_path + '/discriminator')

# saver saves and loads variables of the model to and from checkpoints, 
# which are binary files that maps variable names to tensor values
saver = tf.train.Saver(max_to_keep=10) # maximally save 10 checkpoints

with tf.Session() as sess:  
    # variables need to be initialized before we can use them
    sess.run(tf.initialize_all_variables())
    #print [v.name for v in tf.trainable_variables()] # print all variable names
    
    
    # -------- pre-train discriminator --------
    start = time.time()
    for epoch in range(pre_train_epochs):
        x = get_x(batch_size) # get a batch of real images
        
        d_pre_summary_merge = tf.merge_summary([summary_pre_d_loss, summary_pre_d_x_hist])
        summary_pre_d,_=sess.run([d_pre_summary_merge,pre_optimizer], feed_dict={x_vector: x})
        
        d_writer.add_summary(summary_pre_d, epoch) # add loss summary to tensorboard
        time_lapse = time.time()-start
        start = time.time()
        print "pre-train epoch: ", epoch,", time spent: %.2fs" % time_lapse
   
    print "pre-train done."
    
    
    
    # -------- jointly training discriminator and generator --------
    start = time.time()
    
    # z noise vector that will be used to generate image to check the training progress
    z_sample = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
    
    for epoch in range(training_epochs):
        x = get_x(batch_size) # get a batch of real images
        # mini-batch of noise data from [-1, 1]
        z = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
        
        for i in range(k):
            # Update the discriminator
            d_summary_merge = tf.merge_summary([summary_d_loss, summary_d_x_hist,summary_d_z_hist])
            summary_d,_ = sess.run([d_summary_merge,optimizer_op_d],feed_dict={z_vector:z, x_vector:x}) 
        
        # Update the generator
        _ = sess.run([optimizer_op_g],feed_dict={z_vector:z}) 
        summary_g,_ = sess.run([summary_g_loss,optimizer_op_g],feed_dict={z_vector:z})
        
        if epoch % 10 == 0:
            d_writer.add_summary(summary_d, epoch) # add loss summary to tensorboard
            g_writer.add_summary(summary_g, epoch)
        
        if epoch % 200 == 0 or epoch in [5,20,40,60]:
            time_lapse = time.time()-start
            start = time.time()
            
            print "DCGAN epoch: ", epoch,", time spent: %.2fs" % time_lapse
            g_images = sess.run(g_output,feed_dict={z_vector:z_sample}) # get a generated image
            
            # make a directory for generated images
            if not os.path.exists(train_sample_directory):
                os.makedirs(train_sample_directory)
            
            #Save sample generator images for viewing training progress.
            save_images(images = np.reshape(g_images[0:batch_size_sample],[batch_size_sample,32,32]),\
                        size = [sample_num_rows,sample_num_columns], image_path = train_sample_directory+'/'+str(epoch)+'.png')
        
        if epoch in [0,5,20,60,100,
                    500,1000,2000,3000,4000]:
            # make a directory for trained models
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            
            # save the trained model at different epoch
            saver.save(sess, save_path = model_directory + '/' + str(epoch) + '.cptk')
    print "Done"

### Combine sample images generated from training and save it as GIF

In [ ]:
file_names = sorted([int(f[:-4]) for f in os.listdir(train_sample_directory) if f.endswith('.png')]) # get image filenames
file_names = [str(f)+'.png' for f in file_names] # add the format suffix at the end of the filenames
images = [Image.open(train_sample_directory+f) for f in file_names] # turn into image instances
filename = "train_samples.gif"

if not os.path.exists(train_sample_directory):
            os.makedirs(train_sample_directory)
writeGif(train_sample_directory+filename, images, duration=0.1) # combine images to gif

## Use the trained generator to generate images

In [ ]:
start = time.time()
print 'Loading models...might take a minute'
saver = tf.train.Saver()

with tf.Session() as sess:
    
    # reload the trained model.
    ckpt = tf.train.get_checkpoint_state(model_directory)
    models = ckpt.all_model_checkpoint_paths
    #print 'all saved models: ',ckpt.all_model_checkpoint_paths
    
    # z noise vector that will be used to generate image to check the training progress
    z_sample = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
    
    # loop through each checkpoint and use models to generate images at different training stages
    for index, model in enumerate(models):
        saver.restore(sess, save_path=model)

        g_images = sess.run(g_output,feed_dict={z_vector:z_sample}) # get a generated image

        # make a directory for generated images
        if not os.path.exists(sample_directory):
            os.makedirs(sample_directory)

        #Save generated sample images
        save_images(images = np.reshape(g_images[0:batch_size_sample],[batch_size_sample,32,32]),\
                    size = [sample_num_rows,sample_num_columns], image_path = sample_directory+'/'+str(index)+'.png')

        print "Model "+str(index),': ',model
        
time_lapse = time.time()-start
print " Time spent: %.2fs" % time_lapse

### Combine sample images generated from checkpoints and save it as GIF

<p>Restart the kernel if not working..</p>

In [ ]:
file_names = sorted([int(f[:-4]) for f in os.listdir(sample_directory) if f.endswith('.png')]) # get image filenames
file_names = [str(f)+'.png' for f in file_names] # add the format suffix at the end of the filenames
images = [Image.open(sample_directory+f) for f in file_names] # turn into image instances
filename = "generated_samples.gif"

if not os.path.exists(sample_directory):
            os.makedirs(sample_directory)
writeGif(sample_directory+filename, images, duration=0.1) # combine images to gif

## Interpolation Preparation

In [6]:
model_directory = './dcgan_mnist_4k/models/' # directory to save trained model
sample_inter_directory = './dcgan_mnist_4k/interpolation/' # directory to save the generated images from linear interpolation
# make a directory for generated images
if not os.path.exists(sample_inter_directory):
    os.makedirs(sample_inter_directory)

In [62]:
def concat_interpolation(images, size):
    # get height and width for a single image generated, e.g. 64 * 64
    height, width = images.shape[1], images.shape[2] 
    
    # placeholder for a concatenated img, which have 
    # img_height = image height * num images vertically, e.g. 64 * 5
    # img_width = image width * num images horizontally, e.g. 64 * 5
    img = np.zeros((height * size[0], width * size[1],3))
    #print img.shape

    # loop through each image
    for index, image in enumerate(images):
        j = index / size[0] # image row index
        i = index % size[1] # image column index
        img[j*height:j*height+height, i*width:i*width+width,:] = image
        
        
# spherical interpolation between pointA and pointB, each is a ndarray
# val is a value between [0,1], where 0 returns pointA, 1 returns pointB
def slerp(val, pointA, pointB):
    omega = np.arccos(np.clip(np.dot(pointA/np.linalg.norm(pointA), pointB/np.linalg.norm(pointB)), -1, 1))
    so = np.sin(omega)
    if so == 0:
        return (1.0-val) * pointA + val * pointB # L'Hopital's rule/LERP
    return np.sin((1.0-val)*omega) / so * pointA + np.sin(val*omega) / so * pointB

In [37]:
tf.reset_default_graph()
# size of initial noise vector that will be used for generator
z_size = 100 

# placeholders for inputs into the generator
# shape[0] is None means it could take any integer value
z_vector = tf.placeholder(shape=[None,z_size],dtype=tf.float32, name='z_vectors')

# initialize all parameters of the networks
# weights were initialized from a zero-centered Normal distribution with standard deviation 0.02
# tf.truncated_normal returns random values from a normal distribution and made sure no value exceeds 2 std
initializer = tf.truncated_normal_initializer(stddev=0.02)

# generated mini-batch of images from noisy z vectors
g_output = generator(z_vector)

print 'Loading MNIST models...might take a minute'
saver = tf.train.Saver()

## Interpolation

In [75]:
with tf.Session() as sess:

    # reload the trained model.
    ckpt = tf.train.get_checkpoint_state(model_directory)
    model = ckpt.model_checkpoint_path
    print 'most trained model: ', model

    # z noise vector that will be used to generate image to check the training progress
    z_sample = np.random.uniform(-1.0,1.0,size=[batch_size_sample,z_size]).astype(np.float32)

    # restore model variables
    saver.restore(sess, save_path=model)

    # linear interpolation
    z_sample_one = z_sample[:2]
    #print z_sample_one.shape

    # set fixed input, interpolate from 7 to 9
    sample1='''-0.13812219  0.72763479 -0.12574646 -0.0182618   0.61583346  0.46916214
   0.14667514  0.21866253  0.00216453  0.47648311 -0.77202815  0.68840188
  -0.16999102 -0.54367113 -0.50641257 -0.3653219  -0.84792823  0.75601202
   0.08892749  0.4203513   0.03922871 -0.69616371  0.92312521 -0.28905192
   0.72666872  0.17526712 -0.14319344  0.25685716 -0.26958612 -0.43560231
  -0.83336276 -0.45225969  0.25468764 -0.66877216 -0.111047    0.5109219
   0.36932391  0.63024729 -0.50508118  0.64383519  0.42127076  0.74423504
  -0.5183841   0.13616003  0.41404498 -0.86310548 -0.77108097 -0.80045879
  -0.78332216  0.91736335 -0.08512668 -0.86796314 -0.2598148   0.29403165
   0.02679804  0.62187326  0.61458081  0.7741124   0.86198151 -0.45453411
  -0.60225844  0.22483984 -0.48502383 -0.8646636  -0.08629571  0.5238927
   0.306025    0.85115772 -0.29717201 -0.44797084  0.49626905  0.41648152
   0.04967806  0.58770573 -0.12688731 -0.73914015 -0.31743556 -0.5468775
  -0.6892097  -0.80815262 -0.69773364  0.95089406 -0.3983663   0.8524965
   0.28033748 -0.63092893  0.25420371  0.16844808  0.65201157 -0.18057719
   0.88261253  0.73636746 -0.47242001  0.35281563 -0.64418107 -0.66982579
  -0.33878803 -0.75849366 -0.23950006 -0.45955518'''
    sample2='''0.85101515  0.51374036  0.01500881 -0.42103535  0.24928139 -0.24745837
  -0.12437476 -0.43647423  0.37256166  0.25429964 -0.86342484 -0.50074399
   0.36094844  0.04752741  0.7953878   0.94379628  0.39963263 -0.40967974
  -0.95484734  0.28475836  0.62765479 -0.77327323  0.10163277 -0.64927554
   0.61540842 -0.13411519 -0.44134077  0.07410253  0.21522656  0.3721008
   0.61486959 -0.57751054  0.81080121 -0.20757794  0.6915518   0.50987828
  -0.91912782  0.89409041  0.08876665  0.22706899  0.47728395 -0.88446796
   0.94171816  0.49114835 -0.45817032 -0.38677561  0.98655891  0.13367338
   0.33596939 -0.10619202  0.64132112 -0.74780822  0.67132586  0.24326575
   0.3466444  -0.48140165  0.91027969  0.35116872 -0.76117933 -0.32121259
  -0.65382147  0.86838818 -0.44831353 -0.21466644 -0.24085194  0.89692616
   0.91922706 -0.73745388  0.1747136   0.80469757  0.18904562 -0.19690935
  -0.28799164 -0.242294    0.98938328 -0.61107469  0.6935581  -0.38045514
  -0.74832153 -0.39835727 -0.15829515  0.68855262  0.05665513  0.25415811
   0.85488611 -0.77779335  0.3220959  -0.97219348 -0.49998266  0.60628974
  -0.54538608 -0.32535785  0.27962732 -0.54508322  0.74605769 -0.84807205
  -0.34984127 -0.73864782  0.48457471  0.48459283'''
    # turn the string above into an array. sample1 represents a point in a 100-dimensional space
    sample1 = np.array([float(i.rstrip()) for i in sample1.split(' ') if i != ''])  
    sample2 = np.array([float(i.rstrip()) for i in sample2.split(' ') if i != ''])
    #z_sample_one[1] = np.zeros(100) # keep the second digit constant

    num_inter = 7 # number of interpolated images
    
    '''
    # linear interpolation between sample1 and sample2  
    print 'linear interpolation'
    for i in range(num_inter):
        if i == 0: # interpolation start with point A
            z_sample_one[0] = sample1
        elif i == num_inter-1: # interpolation end with point B
            z_sample_one[0] = sample2
        else:
            z_sample_one[0] += (sample2-sample1)/num_inter # linear interpolate from point A to B
        #print z_sample_one,'\n'

        g_images = sess.run(g_output,feed_dict={z_vector:z_sample_one})
        g_image = g_images[0] # only retrieve the first returned image
        
        
        # save interpolated image
        image = inverse_transform(g_image)
        image = np.reshape(image,[32,32])
        image = Image.fromarray(image)
        scipy.misc.imsave(sample_inter_directory+'/'+str(i)+'.png', image)
        
    '''
    print 'spherical interpolation'
    # spherical interpolation between pointA and pointB, each is a ndarray
    # val is a value between [0,1], where 0 returns pointA, 1 returns pointB
    for i, value in enumerate(np.linspace(0,1,num_inter)):
        z_sample_one[0] = slerp(val=value, pointA=sample1, pointB=sample2)
        #print z_sample_one,'\n'

        g_images = sess.run(g_output,feed_dict={z_vector:z_sample_one})
        g_image = g_images[0] # only retrieve the first returned image
        
        # save interpolated image
        image = inverse_transform(g_image)
        image = np.reshape(image,[32,32])
        image = Image.fromarray(image)
        scipy.misc.imsave(sample_inter_directory+'/'+str(i)+'.png', image)
    
    print 'done'

most trained model:  ./dcgan_mnist_4k/models/4000.cptk
spherical interpolation
done


### turn the interpolate images into GIF

In [61]:
file_names = sorted([int(f[:-4]) for f in os.listdir(sample_inter_directory) if f.endswith('.png')]) # get image filenames
file_names = [str(f)+'.png' for f in file_names] # add the format suffix at the end of the filenames
images = [Image.open(sample_inter_directory+f) for f in file_names] # turn into image instances
filename = "aa.gif"
writeGif(sample_inter_directory+filename, images, duration=0.1) # combine images to gif